In [ ]:
using MakieCore
import MakieCore: convert_arguments 

# This part would be the extension package (calls like lines, series, etc.)
using CairoMakie

import NamedTrajectories as NT

In [ ]:
traj = rand(NT.NamedTrajectory, 10);

- We can also define convert_arguments for a series, but series is in Makie, not MakieCore

In [ ]:
function convert_arguments(
    P::Type{<:Makie.Series}, 
    traj::NT.NamedTrajectory,
    name::Symbol;
    transformation::Union{Function, Nothing}=nothing
)
    if !isnothing(transformation)
        output_data = try
            stack(transformation.(eachcol(traj[name])))
        catch
            throw(ArgumentError("Transformation of $(name) failed."))
        end
    else
        output_data = traj[name]
    end

    return convert_arguments(P, NT.get_times(traj), output_data)
end

MakieCore.used_attributes(::Type{<:Makie.Series}, ::NT.NamedTrajectory, ::Symbol) = (:transformation,)
MakieCore.plottype(::NT.NamedTrajectory, ::Symbol) = Makie.Series

In [ ]:
# @recipe(PlotComponents, traj, components) do scene
#     Attributes(
#         color=:seaborn_colorblind6,
#         linestyle=theme(scene, :linestyle),
#         linewidth=theme(scene, :linewidth),
#         marker=nothing,
#         markersize=nothing,
#     )
# end

In [ ]:
function my_plot()
    f = Figure()
    plot(
        f[1,1], traj, :x, 
        transformation=x -> [abs.(x); x .^ 2],
        color=:seaborn_colorblind6,
        marker=:circle,
        markersize=5,
    )
    return f
end

In [ ]:
my_plot()

In [ ]:
with_theme(my_plot, theme_minimal())

In [ ]:
f = Figure()
ax = Axis(f[1, 1])
plot!(ax, traj, :x, linewidth=3)
Legend(f[1, 2], ax)
plot(f[2, 1], traj, :u)
f